In [2]:
import pandas as pd

In [8]:
# Load data
df = pd.read_csv("../data/processed/labelled_texts.csv")
df = df[['complaint_what_happened', 'topic']]
df.head()

,complaint_what_happened,topic
0,Good morning my name is XXXX XXXX and I apprec...,Bank Account services
1,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card
2,Chase Card was reported on XX/XX/2019. However...,Credit card or prepaid card
3,"On XX/XX/2018, while trying to book a XXXX XX...",Credit card or prepaid card
4,my grand son give me check for {$1600.00} i de...,Bank Account services


In [20]:
type(df.loc[0, 'complaint_what_happened'])

str

# Dataset Class

In [33]:
from torch.utils.data import Dataset

class CustomerComplaintsDataset(Dataset):
    def __init__(self, csv_path: str) -> None:
        super().__init__()
        self.df = pd.read_csv(csv_path)
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        text = self.df.loc[idx, 'complaint_what_happened']
        label = self.df.loc[idx, 'topic']
        return text, label

In [34]:
from torch.utils.data import DataLoader
dataset = CustomerComplaintsDataset("../data/processed/labelled_texts.csv")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [35]:
next(iter(dataloader))

[('On XX/XX/18, opened two accounts, one savings, one checking at Chase bank branch in Washignton, DC. At time of opening, a new account bonus code was provided, which banker said was invalid. Banker advised that I could obtain a new code, provide it within 10 days, and it would be no problem to apply it to the account. The code offers a financial incentive if certain conditions are met. On XX/XX/18, provided via email a new bonus code and was advised on XX/XX/18 by banker code was invalid. Upon review, I noticed a typographical error in the emailed bonus code and immediately replied to banker with the correct code and the photo of the code. To date, I have not received a response.',
  'On XXXX XXXX,2017 I called Amazon chase company asked what the pay off was. They gave me the amount in which I paid ofc account. I then receive a bill for a interest payment for the next month. When card was paid off.',
  'On XX/XX/2014 Chase Banking copy, s of my bank statements a loan That I know noth